# <font color = "blue">**Task2: Restaurant Recommendation**

# <font color = "black">**Importing necessary libraries**</font>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
import warnings
warnings.filterwarnings('ignore')

# <font color = black> **Load and preprocess dataset**

In [ ]:
df = pd.read_csv('/content/Dataset .csv')  # Replace with your actual file path
df = df.dropna(subset=['Cuisines', 'Aggregate rating', 'City', 'Restaurant Name'])

In [ ]:
# Clean and tokenize cuisines
df['Cuisines'] = df['Cuisines'].str.lower().str.strip()

#**TF-IDF vectorization of 'Cuisines'**

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['Cuisines'])

 # <font color = blue>**Process structured features**

In [ ]:
structured_features = df[['Price range']].copy()
structured_features['Online delivery'] = df['Has Online delivery'].map({'Yes': 1, 'No': 0}).fillna(0)

In [ ]:
# Normalize numerical features
scaler = StandardScaler()
structured_scaled = scaler.fit_transform(structured_features)

# **Combine features**

In [ ]:
restaurant_vectors = hstack([tfidf_matrix, structured_scaled])

# **Recommendation Function**

In [ ]:
def recommend_restaurants(cuisine_input, price_range_input, online_delivery_input, city_input, top_n=5):
    # Filter by city
    df_filtered = df[df['City'].str.lower().str.strip() == city_input.lower().strip()]

    if df_filtered.empty:
        print("No restaurants found in the specified city.")
        return pd.DataFrame()

    # TF-IDF vectorizer & scaler (use fit from full dataset)
    cuisine_vectors = tfidf.transform(df_filtered['Cuisines'])
    structured_features = df_filtered[['Price range']].copy()
    structured_features['Online delivery'] = df_filtered['Has Online delivery'].map({'Yes': 1, 'No': 0}).fillna(0)
    structured_scaled = scaler.transform(structured_features)

    # Combine vectors
    restaurant_vectors = hstack([cuisine_vectors, structured_scaled])

    # User vector
    user_cuisine_vec = tfidf.transform([cuisine_input.lower().strip()])
    user_struct = scaler.transform([[price_range_input, 1 if online_delivery_input.lower() == 'yes' else 0]])
    user_vector = hstack([user_cuisine_vec, user_struct])

    # Cosine similarity
    similarity = cosine_similarity(user_vector, restaurant_vectors).flatten()

    # Top N indices
    top_indices = similarity.argsort()[-top_n:][::-1]
    recommended = df_filtered.iloc[top_indices][[
        'Restaurant Name', 'City', 'Cuisines', 'Price range',
        'Has Online delivery', 'Aggregate rating'
    ]].copy()

    recommended['Similarity Score'] = similarity[top_indices]
    recommended = recommended.reset_index(drop=True)

    # Add Serial Number starting from 1
    recommended.index = recommended.index + 1
    recommended.index.name = "S.No"

    return recommended



# <font color = blue> **Example Usage**

In [ ]:
# recommendations = recommend_restaurants(
#     cuisine_input='north indian, chinese',
#     price_range_input=2,
#     online_delivery_input='Yes',
#     top_n=5
# )

# print("Top 5 Recommended Restaurants:\n")
# recommendations

recommendations = recommend_restaurants(
    cuisine_input='chinese',
    price_range_input=2,
    online_delivery_input='yes',
    city_input='Bangalore',
    top_n=5
)
recommendations


,Restaurant Name,City,Cuisines,Price range,Has Online delivery,Aggregate rating,Similarity Score
S.No,,,,,,,
1,Truffles,Bangalore,"american, burger, cafe",2,Yes,4.7,0.746160
2,Onesta,Bangalore,"pizza, cafe, italian",2,Yes,4.3,0.746160
3,Eat Street,Bangalore,"north indian, chinese, italian, street food, d...",1,Yes,4.3,0.697975
4,Bombay Brasserie,Bangalore,modern indian,3,Yes,4.2,0.674383
5,Sultans of Spice,Bangalore,"north indian, mughlai",3,Yes,4.1,0.674383


In [ ]:
recommendations = recommend_restaurants(
    cuisine_input='North Indian, Mughlai',
    price_range_input=2,
    online_delivery_input='yes',
    city_input='Aurangabad',
    top_n=5
)
recommendations

,Restaurant Name,City,Cuisines,Price range,Has Online delivery,Aggregate rating,Similarity Score
S.No,,,,,,,
1,Great Sagar Restaurant,Aurangabad,"north indian, mughlai",2,No,3.3,0.019810
2,Madhuban Restaurant - Welcome Hotel Rama Inter...,Aurangabad,"north indian, chinese, continental, italian, m...",4,No,3.4,-0.050724
3,Kareem's Kabab & Biryani,Aurangabad,mughlai,2,No,3.4,-0.073618
4,Downside Up,Aurangabad,"cafe, mughlai, north indian",2,No,3.2,-0.083890
5,Kream N Krunch,Aurangabad,"north indian, chinese, continental, italian",3,No,3.6,-0.128503


# **Report: Restaurant Recommendation System**

<h3><u>Objective:</u></h3>
<p>To build a content-based filtering recommendation system that suggests restaurants based on user preferences for <b>cuisines</b>, <b>price range</b> etc.</p>

<h3><u>Data Preprocessing:</u></h3>
<ul>
  <li>Removed missing values in critical columns like <b>Cuisines</b>, <b>City</b>, and <b>Aggregate rating</b>.</li>
  <li>Standardized text and mapped delivery options to numeric values.</li>
</ul>

<h3><u>Feature Engineering:</u></h3>
<ul>
  <li><b>Cuisines</b> - Vectorized using <b>TF-IDF</b>.</li>
  <li><b>Price Range & Online Delivery</b> - Scaled using <b>StandardScaler</b>.</li>
  <li><b>City</b> - One-hot encoded and included in the similarity score.</li>
</ul>

<h3><u>Recommendation Approach:</u></h3>
<p>User input is converted into a feature vector and compared to all restaurants using <b>cosine similarity</b>. The most similar restaurants are returned, further sorted by <b>Aggregate Rating</b>.</p>

<h3><u>User Input Parameters:</u></h3>
<ul>
  <li><b>Cuisine:</b> e.g., "north indian, chinese"</li>
  <li><b>Price Range:</b> 1 to 4</li>
  <li><b>Online Delivery:</b> Yes / No</li>
  <li><b>City:</b> e.g., "Delhi", "Bangalore"</li>
</ul>

<h3><u>Output:</u></h3>
<p>Top 5 recommended restaurants with:</p>
<ul>
  <li><b>Restaurant Name</b></li>
  <li><b>City</b></li>
  <li><b>Cuisines</b></li>
  <li><b>Price Range</b></li>
  <li><b>Online Delivery</b></li>
  <li><b>Aggregate Rating</b></li>
  <li><b>Similarity Score</b></li>
</ul>


<h6><b>Built using pandas, scikit-learn, TFIDF, and cosine similarity.</b></h6>
